In [2]:
import boto3
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
ses_client = boto3.client(
    'ses',
    region_name=os.getenv('AWS_REGION'),
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
)

In [ ]:
from premailer import transform # <-- 1. IMPORTAR PREMAILER
user_name = "Fernando"
html_template = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <style>
        body {{ font-family: Arial, sans-serif; margin: 0; padding: 0; background-color: #f4f7f6; }}
        .container {{ max-width: 600px; margin: 20px auto; background-color: #ffffff; border: 1px solid #e0e0e0; border-radius: 12px; overflow: hidden; }}
        .header {{ background-color: #003366; color: white; padding: 30px; text-align: center; }}
        .header h1 {{ margin: 0; font-family: 'Montserrat', sans-serif; font-size: 28px; }}
        .content {{ padding: 30px; color: #333; line-height: 1.7; font-size: 16px; }}
        .content a {{ color: #003366; font-weight: bold; text-decoration: none; }}
        .cta-section {{ text-align: center; padding: 20px 30px; background-color: #f9f9f9; border-top: 1px solid #eee; }}
        .button {{ display: inline-block; background-color: #d35400; color: white !important; padding: 12px 25px; border-radius: 50px; font-weight: bold; text-decoration: none; font-size: 16px; }}
        .footer {{ text-align: center; padding: 20px; font-size: 12px; color: #888; }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header"><h1>¡Bienvenido a bordo de ✨igles-IA!</h1></div>
        <div class="content">
            <p>Hola {user_name},</p>
            <p>Tu suscripción se ha completado con éxito. ¡Estamos muy contentos de que formes parte de nuestra comunidad!</p>
            <p>Desde el próximo lunes, recibirás en tu correo el resumen semanal con las enseñanzas del Papa León XIV.</p>
        </div>
        <div class="cta-section">
            <p>¿No puedes esperar? Echa un vistazo al último resumen publicado:</p>
            <a href="https://igles-ia.es/resumenes" class="button">Leer el último resumen</a>
        </div>
        <div class="footer">
            <p>Conoce nuestro proyecto: https://igles-ia.es/sobre-nosotros.<br>
            <p>Recibes este correo porque te has suscrito en igles-ia.es.<br>
            También puedes explorar nuestro <a href="https://igles-ia.es/resumenes.html">archivo histórico</a>.</p>
        </div>
    </div>
</body>
</html>
"""

# --- 2. LA MAGIA OCURRE AQUÍ ---
# Premailer coge tu HTML y CSS y los combina en un HTML con estilos en línea
html_inlined = transform(html_template)

In [23]:
FROM_EMAIL = 'igles-IA@igles-ia.es' # Tu email verificado en SES



ses_client.send_email(
    Source=f"Igles-IA <{FROM_EMAIL}>",
    Destination={
        'ToAddresses': ["igles.ia.newsletter@gmail.com"],
    },
    Message={
        'Subject': {
            'Data': "✨Igles-IA | León XIV: Newsletter semanal de discursos y homilias."
        },
        'Body': {
            'Html': {
                'Data': html_inlined
            }
        }
    }
)

{'MessageId': '01020197cb8700ac-d2a6bba5-4df1-4f04-9c44-fc8522084bee-000000',
 'ResponseMetadata': {'RequestId': '73ef229e-fa8d-4311-ab5c-6f55f46a5779',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 02 Jul 2025 14:25:11 GMT',
   'content-type': 'text/xml',
   'content-length': '326',
   'connection': 'keep-alive',
   'x-amzn-requestid': '73ef229e-fa8d-4311-ab5c-6f55f46a5779'},
  'RetryAttempts': 0}}

In [ ]:
import boto3
import os

# Inicializar el cliente de SES (servicio de email de AWS)
ses_client = boto3.client('ses', region_name=os.environ['AWS_REGION'])
FROM_EMAIL = 'igles-IA@igles-ia.es' # Tu email verificado en SES

def lambda_handler(event, context):
    # Extraer los datos del usuario del "evento" que envía Cognito
    user_email = event['request']['userAttributes']['email']
    user_name = event['request']['userAttributes'].get('name', 'amigo/a') # Coge el nombre, o usa un fallback

    # Definir el contenido del email
    subject = "¡Bienvenido a bordo de Igles-IA!"
    body_html = f"""
        <html>
        <body>
            <h1>¡Hola {user_name}, te damos la bienvenida a Igles-IA!</h1>
            <p>Tu suscripción se ha completado con éxito. A partir del próximo lunes, recibirás en tu correo ({user_email}) el resumen semanal con las enseñanzas del Papa León XIV.</p>
            <p>Mientras tanto, puedes explorar nuestro <a href="https://igles-ia.es/resumenes.html">archivo de resúmenes anteriores</a>.</p>
            <p>¡Que Dios te bendiga!</p>
        </body>
        </html>
    """

    # Enviar el email usando SES
    try:
        response = ses_client.send_email(
            Source=FROM_EMAIL,
            Destination={'ToAddresses': [user_email]},
            Message={
                'Subject': {'Data': subject},
                'Body': {'Html': {'Data': body_html}}
            }
        )
        print(f"Email de bienvenida enviado a {user_email}. Message ID: {response['MessageId']}")
    except Exception as e:
        print(f"Error al enviar email a {user_email}: {e}")

    # Es importante devolver el objeto 'event' a Cognito
    return event

In [3]:
# SES
def send_email(to_address, subject, body):
    ses_client = boto3.client(
        'ses',
        region_name=os.getenv('AWS_REGION'),
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
    )

    response = ses_client.send_email(
        Source=os.getenv('SES_SOURCE_EMAIL'),
        Destination={
            'ToAddresses': [to_address]
        },
        Message={
            'Subject': {
                'Data': subject,
                'Charset': 'UTF-8'
            },
            'Body': {
                'Text': {
                    'Data': body,
                    'Charset': 'UTF-8'
                }
            }

        }

        

SyntaxError: incomplete input (2765329122.py, line 29)